<a href="https://colab.research.google.com/github/oneoftwo/CH453_AI_chemistry/blob/main/practice_3/practice_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Practice Session 3**

In [13]:
# DO NOT CHANGE!
!rm -rf CH453*
!git clone https://github.com/oneoftwo/CH453_AI_chemistry/
!ls
!mv ./CH453_AI_chemistry/* ./
!ls
!pip install rdkit

import rdkit
print(rdkit.__version__)

Cloning into 'CH453_AI_chemistry'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 100 (delta 38), reused 4 (delta 1), pack-reused 0
Receiving objects: 100% (100/100), 1.76 MiB | 5.13 MiB/s, done.
Resolving deltas: 100% (38/38), done.
assignment_1	    data	practice_1  README.md
CH453_AI_chemistry  practice_0	practice_3  sample_data
mv: cannot move './CH453_AI_chemistry/assignment_1' to './assignment_1': Directory not empty
mv: cannot move './CH453_AI_chemistry/data' to './data': Directory not empty
mv: cannot move './CH453_AI_chemistry/practice_0' to './practice_0': Directory not empty
mv: cannot move './CH453_AI_chemistry/practice_1' to './practice_1': Directory not empty
mv: cannot move './CH453_AI_chemistry/practice_3' to './practice_3': Directory not empty
assignment_1	    data	practice_1  README.md
CH453_AI_chemistry  practice_0	practice_3  sample_data
2023.03.3


# **Model**

In [14]:
import torch
from torch import nn
from torch.nn import functional as F


class LinearRegressor(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        super().__init__()
        # initialize model
        self.fc_1 = nn.Linear(in_dim, hid_dim)
        self.dropout = nn.Dropout(0.2)
        self.fc_2 = nn.Linear(hid_dim, out_dim)

    def forward(self, x):
        # define model forward (conduct tensor operation)
        x = self.fc_1(x) # [bs, in_dim] --> [bs, hid_dim]
        x = F.relu(x) # activation
        x = self.dropout(x)
        x = self.fc_2(x) # [bs, hid_dim] --> [bs, out_dim] # in this case 1
        return x.squeeze()



# **Dataset**

In [15]:
import torch
from torch.utils.data import Dataset
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors


class LogPDataset(Dataset):
    def __init__(self, smiles_list):
        super().__init__()
        # initialize dataset
        self.smiles_list = smiles_list

    def __getitem__(self, idx):
        # get item from index
        smiles = self.smiles_list[idx]
        out_dict = {}
        try:
            out_dict["logp"] = torch.Tensor([self.smiles_to_logp(smiles)]).squeeze()
            out_dict["fp"] = self.smiles_to_fp(smiles)
            out_dict["SMILES"] = smiles
        except:
            print(smiles)
        return out_dict

    def __len__(self):
        # return the legnth of the total dataset
        pass
        return len(self.smiles_list)

    def smiles_to_logp(self, smiles):
        mol = Chem.MolFromSmiles(smiles)
        return Descriptors.MolLogP(mol)

    def smiles_to_fp(self, smiles):
        # make fp (torch.Tensor) from SMILES.
        # nBits (the size of FP is conventionally 2048, but here for training purpose we set to 512)
        mol = Chem.MolFromSmiles(smiles)
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=512)
        fingerprint_tensor = torch.tensor(list(fingerprint), dtype=torch.float32)
        return fingerprint_tensor


temp_set = LogPDataset(["CCC", "CCO"])
print(temp_set[1])


{'logp': tensor(-0.0014), 'fp': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        

# **Train and Validate**

In [16]:
import torch
from torch.nn import functional as F
from tqdm.notebook import tqdm

def single_epoch(model, data_loader, optimizer=None):
    # trains model for single epoch with given data_loader
    total_loss = 0.0

    for batch_sample in tqdm(data_loader):
        fp = batch_sample["fp"]
        logp = batch_sample["logp"]

        pred_logp = model(fp.cuda())
        loss = F.mse_loss(pred_logp, logp.cuda(), reduction="sum")

        if optimizer != None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item()

    result_dict = {}
    result_dict["loss"] = total_loss / len(data_loader.dataset)

    return result_dict




In [17]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, random_split
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)


# read SMILES file
smiles_fn = "./data/PubChem_10K.txt"
with open(smiles_fn, 'r') as file:
    smiles_list = [line.strip().split()[1] for line in file]

# define dataset
all_set = LogPDataset(smiles_list)

# split dataset into train / valid / test (conventionaly, 8:1:1)
train_set, valid_set, test_set = random_split(all_set, [0.8, 0.1, 0.1])
print(f"length of train, valid, and test set: {len(train_set), len(valid_set), len(test_set)}")

# define data loader which batches dataset
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)


# define model
model = LinearRegressor(512, 256, 1)
model.cuda()
print(model)

# set optimizer (Adam is the most frequently used optimizer)
# 3e-3 is a quite high value for lr, please lower it while assignments (1e-3 ~ 1e-6)
optimizer = optim.Adam(model.parameters(), lr=3e-3)

# train
train_loss_history, valid_loss_history = [], []
for epoch_idx in range(1, 26):
    model.train()
    tr = single_epoch(model, train_loader, optimizer)
    train_loss_history.append(tr["loss"])

    with torch.no_grad():
        model.eval()``
        vr = single_epoch(model, valid_loader)
        valid_loss_history.append(vr["loss"])

    torch.save(model.state_dict(), f"./model_{epoch_idx}.pt")

    print(epoch_idx)
    print(tr)
    print(vr)
    print()


length of train, valid, and test set: (8000, 1000, 1000)
LinearRegressor(
  (fc_1): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc_2): Linear(in_features=256, out_features=1, bias=True)
)


  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

1
{'loss': 7.0569806518554685}
{'loss': 5.647322143554687}



  0%|          | 0/63 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

24
{'loss': 0.48813632893562314}
{'loss': 3.477001419067383}



  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

25
{'loss': 0.49009783506393434}
{'loss': 3.484009338378906}



In [18]:
# plot the results
from matplotlib import pyplot as plt


plt.plot(train_loss_history, label='Train Loss')
plt.plot(valid_loss_history, label='Validation Loss')
plt.legend(loc='upper right')
plt.show()


In [19]:
# test with the best model (with lowest validation loss)

# load model
model_fn = "./model_20.pt" # choose the best model
model.load_state_dict(torch.load(model_fn, map_location="cpu"))
model.cuda()
model.eval()

# make logp predictions
all_logps, all_pred_logps, all_smiles = [], [], []
with torch.no_grad():
    for batch_sample in test_loader:
        fp = batch_sample["fp"]
        logp = batch_sample["logp"]
        smiles_list = batch_sample["SMILES"]

        all_smiles += smiles_list
        all_logps += logp.cpu().tolist()

        pred_logp = model(fp.cuda()) #
        all_pred_logps += pred_logp.cpu().tolist()


import pandas as pd
from matplotlib import pyplot as plt


pd.DataFrame({"SMILES":all_smiles, "pred_logp":all_pred_logps, "logp":all_logps})
plt.scatter(all_logps, all_pred_logps, s=10)
plt.xlabel('True LogP')
plt.ylabel('Pred LogP')
plt.plot(all_logps, all_logps, label='y=x', color='red', linestyle='--')
# Display the plot
plt.show()

